# Approche Deep Learning

Nous proposons une approche élégante en utilisant les réseaux de neurones convolutionnels directements appliqués aux images issues des plot des landmarks.
Cette approche élégante sera conduite de manière la plus optimisée possible.

Il y a aura  deux types d'images :
- les plot avec des points noirs
- les plot avec les points colorés délimitant les différentes zones du visage (annotation manuelle des images)

Ce notebook comprendra une approche directe (pas d'augmentation, pas/peu d'optimisation d'hyperparametres) et un approche avancée (data augmentation, architecture optimisée) et si le temps le permet du fine tuning ou transfert learning vers d'autres backbones plus avancés.

## Librairies

In [6]:
from keras.models import Sequential
from keras.layers import Dense, Conv2D, Flatten, MaxPool2D
from keras.preprocessing.image import ImageDataGenerator
from scikeras.wrappers import KerasClassifier

### Data generation and preprocessing

In [7]:
# Data Preparation
train_datagen =         ImageDataGenerator(rescale=1./255)
test_datagen =          ImageDataGenerator(rescale=1./255)

train_generator =       train_datagen.flow_from_directory(
                        r'../CK+_lands/images/train/',
                        target_size=(150, 150),
                        batch_size=32,
                        class_mode='categorical')

validation_generator =  test_datagen.flow_from_directory(
                        r'../CK+_lands/images/val/',
                        target_size=(150, 150),
                        batch_size=32,
                        class_mode='categorical')

Found 302 images belonging to 6 classes.


Found 15 images belonging to 6 classes.


### Building / Compiling / Fitting model

In [8]:
# Model Building
model =     Sequential()
model.add(Conv2D(32, kernel_size=(3, 3), activation='relu', input_shape=(150, 150, 3)))
model.add(MaxPool2D(pool_size=2, strides=2, padding="valid"))
model.add(Conv2D(64, kernel_size=(3, 3), activation='relu'))
model.add(MaxPool2D(pool_size=2, strides=2, padding="valid"))
model.add(Conv2D(128, kernel_size=(3, 3), activation='relu'))
model.add(MaxPool2D(pool_size=2, strides=2, padding="valid"))
model.add(Conv2D(256, kernel_size=(3, 3), activation='relu'))
model.add(MaxPool2D(pool_size=2, strides=2, padding="valid"))
model.add(Flatten())
model.add(Dense(256, activation='relu'))
model.add(Dense(6, activation='softmax'))

# Compilation
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

model.summary()

Model: "sequential_3"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_12 (Conv2D)          (None, 148, 148, 32)      896       
                                                                 
 max_pooling2d_12 (MaxPooli  (None, 74, 74, 32)        0         
 ng2D)                                                           
                                                                 
 conv2d_13 (Conv2D)          (None, 72, 72, 64)        18496     
                                                                 
 max_pooling2d_13 (MaxPooli  (None, 36, 36, 64)        0         
 ng2D)                                                           
                                                                 
 conv2d_14 (Conv2D)          (None, 34, 34, 128)       73856     
                                                                 
 max_pooling2d_14 (MaxPooli  (None, 17, 17, 128)      

### Fitting model

In [10]:
model.fit(train_generator, epochs=20, validation_data=validation_generator)

Epoch 1/20
10/10 [==============================] - 6s 578ms/step - loss: 0.2666 - accuracy: 0.8974 - val_loss: 0.6636 - val_accuracy: 0.8667
Epoch 2/20
10/10 [==============================] - 6s 592ms/step - loss: 0.2341 - accuracy: 0.9106 - val_loss: 0.5250 - val_accuracy: 0.8667
Epoch 3/20
10/10 [==============================] - 6s 591ms/step - loss: 0.1264 - accuracy: 0.9603 - val_loss: 0.6508 - val_accuracy: 0.9333
Epoch 4/20
10/10 [==============================] - 6s 596ms/step - loss: 0.0973 - accuracy: 0.9636 - val_loss: 0.7816 - val_accuracy: 0.8667
Epoch 5/20
10/10 [==============================] - 6s 609ms/step - loss: 0.0308 - accuracy: 0.9934 - val_loss: 0.7510 - val_accuracy: 0.8667
Epoch 6/20
10/10 [==============================] - 6s 605ms/step - loss: 0.0267 - accuracy: 0.9934 - val_loss: 1.0351 - val_accuracy: 0.8000
Epoch 7/20
10/10 [==============================] - 6s 603ms/step - loss: 0.0145 - accuracy: 0.9967 - val_loss: 1.0906 - val_accuracy: 0.8667
Epoch 

In [11]:
model.save('emotion_classifier_model.h5')  

c:\Work\Deep-RL-class\deep-rl-class-main\deep-rl-class-main\.conda\lib\site-packages\keras\src\engine\training.py:3000: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


In [12]:
test_datagen =          ImageDataGenerator(rescale=1./255)

test_generator =        train_datagen.flow_from_directory(
                        r'../CK+_lands/images/test/',
                        target_size=(150, 150),
                        batch_size=32,
                        class_mode='categorical')

evaluation =            model.evaluate(test_generator)

print("Test Loss:", evaluation[0])
print("Test Accuracy:", evaluation[1])

Found 15 images belonging to 6 classes.
1/1 [==============================] - 0s 230ms/step - loss: 0.2448 - accuracy: 0.8667
Test Loss: 0.2448122352361679
Test Accuracy: 0.8666666746139526


### Tuning the model ( à voir si on le fait, pcq celui de base est très bon)

In [ ]:
from sklearn.model_selection import GridSearchCV

In [ ]:
def create_model(layers):
    model =     Sequential()
    model.add(Conv2D(32, kernel_size=(3, 3), activation='relu', input_shape=(150, 150, 3)))
    model.add(Conv2D(64, kernel_size=(3, 3), activation='relu'))
    model.add(Conv2D(128, kernel_size=(3, 3), activation='relu'))
    model.add(Flatten())
    
    for i, nodes in enumerate(layers):
        model.add(Dense(units=nodes, activation='relu'))
            
    model.add(Dense(units=6, activation='softmax'))
    
    model.compile(optimizer = 'Adam', loss = 'binary_crossentropy', metrics = ['accuracy'])
    return model

In [22]:
layers = [[128], (256, 128), (200, 150, 120)]
param_grid = dict(layers=layers)
model = KerasClassifier(model=create_model, verbose=1, layers=layers)
grid = GridSearchCV(estimator=model, param_grid=param_grid, verbose=1)
X_train, y_train = next(train_generator)
grid_results = grid.fit(X=X_train, y=y_train, validation_data=(next(validation_generator)))

Fitting 5 folds for each of 3 candidates, totalling 15 fits
1/1 [==============================] - 1s 725ms/step


KeyboardInterrupt: 

In [ ]:
print("Best: {0}, using {1}".format(grid_results.best_score_, grid_results.best_params_))
means = grid_results.cv_results_['mean_test_score']
stds = grid_results.cv_results_['std_test_score']
params = grid_results.cv_results_['params']
for mean, stdev, param in zip(means, stds, params):
    print('{0} ({1}) with: {2}'.format(mean, stdev, param))